In [119]:
import pybitflyer
import json
import time
from threading import Thread
import threading
import os
import pandas as pd
import talib
import matplotlib.pyplot as plt
import datetime
from IPython.display import display, clear_output
import requests
import numpy as np
import xgboost 
from sklearn.model_selection import train_test_split, GridSearchCV
import copy
import datetime
import pickle
%matplotlib inline

In [120]:
# -------------------------------------

end_point = 'wss://ws.lightstream.bitflyer.com/json-rpc'

public_channels = ['lightning_executions_FX_BTC_JPY','lightning_board_snapshot_FX_BTC_JPY']
private_channels = ['child_order_events', 'parent_order_events']
# -------------------------------------

In [ ]:
#59分問題解決？
for d in range(7,20):
    ohlc2 = pd.read_csv('scraping-bot/2020-08-'+str(d).zfill(2)+'-min.csv')
    ohlc3 = pd.read_csv('scraping-bot/2020-08-'+str(d+1).zfill(2)+'-min.csv')
    
    pb = pybitflyer.API(api_key=key,api_secret=secret)
    executions = pb.executions(product_code='FX_BTC_JPY')
    date = (dateutil.parser.parse(executions[-1]['exec_date']) + timedelta(hours=9))
    last_id = 10**10
    last_id = ohlc3['start-id'].values[0]
    while True:
        df_day = pd.DataFrame(columns=['start-id','end-id','timestamp','Open','High','Low','Close','Volume','day','month','year'])
        while True:
            df_minute = pd.DataFrame(columns=['id','side','price','size','timestamp','minute','day','month','year'])
            while True:
                time.sleep(0.6)
                r = pb.executions(product_code='FX_BTC_JPY',count=500,before=last_id)
                if type(r) != list:
                    print(r)
                    time.sleep(60)
                else:
                    executions = r[::-1]
                    df_executions = pd.DataFrame(executions)
                    df_executions['timestamp'] = df_executions['exec_date'].apply(lambda x: int((dateutil.parser.parse(x)).timestamp()))
                    df_executions['minute'] = df_executions['exec_date'].apply(lambda x: (dateutil.parser.parse(x)+timedelta(hours=9)).minute)
                    df_executions['day'] = df_executions['exec_date'].apply(lambda x: (dateutil.parser.parse(x)+timedelta(hours=9)).day)
                    df_executions['month'] = df_executions['exec_date'].apply(lambda x: (dateutil.parser.parse(x)+timedelta(hours=9)).month)
                    df_executions['year'] = df_executions['exec_date'].apply(lambda x: (dateutil.parser.parse(x)+timedelta(hours=9)).year)
                    df_executions = df_executions[['id','side','price','size','timestamp','minute','day','month','year']]
                    df_minute = pd.concat([df_executions,df_minute],ignore_index=True)

                    last_id = df_minute['id'].values[0]

                    if df_minute['minute'].nunique() >= 2:
                        df_minute = df_minute[df_minute['minute'] == df_minute['minute'].values[-1]]
                        #無駄に捨てすぎないように再利用
                        last_id = df_minute['id'].values[0]
                        minute_record = pd.DataFrame({'start-id':[df_minute['id'].values[0]],
                                                                                 'end-id':[df_minute['id'].values[-1]],
                                                                                 'timestamp':[int(df_minute['timestamp'].mean())-(int(df_minute['timestamp'].mean())%60)],
                                                                                 'Open':[df_minute['price'].values[0]],
                                                                                 'High':[df_minute['price'].max()],
                                                                                 'Low':[df_minute['price'].min()],
                                                                                 'Close':[df_minute['price'].values[-1]],
                                                                                'Volume':[df_minute['size'].sum()],
                                                                                'day':[df_minute['day'].values[0]],
                                                                                'month':[df_minute['month'].values[0]],
                                                                                'year':[df_minute['year'].values[0]]})
                        df_day = pd.concat([minute_record,df_day],ignore_index=True)
                        break

            print(df_day.shape)
            break
            if df_day['day'].nunique() >= 2:
                df_day = df_day[df_day['day'] == df_day['day'].values[-1]]
                last_id = df_day['start-id'].values[0]
                day = str(df_day['day'].values[0]).zfill(2)
                month =str(df_day['month'].values[0]).zfill(2)
                year = str(df_day['year'].values[0])
                df_day = df_day[['start-id','end-id','timestamp','Open','High','Low','Close','Volume']]
                df_day.to_csv(year+'-'+month+'-'+day+'-'+'min.csv',index=False)
                break
        break
    print('now')
    ohlc2 = pd.concat([ohlc2,df_day.tail(1)[['start-id','end-id','timestamp','Open','High','Low','Close','Volume']]],ignore_index=True)
    ohlc2.to_csv('scraping-bot/2020-08-'+str(d).zfill(2)+'-min.csv',index=False)

In [1]:
import pandas as pd

In [184]:
ps = pd.DataFrame(columns =['price','size'])
ps = ps.append(pd.Series([110,0.1],index=['price','size']),ignore_index=True)
ps = ps.append(pd.Series([111,0.2],index=['price','size']),ignore_index=True)
ps = ps.append(pd.Series([112,0.4],index=['price','size']),ignore_index=True)
ps = ps.append(pd.Series([113,0.8],index=['price','size']),ignore_index=True)

In [190]:
ps

,price,size,cumsum,cumsum2
0,110.0,0.1,0.1,0.0
1,111.0,0.2,0.3,0.1
2,112.0,0.4,0.7,0.3
3,113.0,0.8,1.5,0.7


In [187]:
ps['cumsum'] = ps['size'].cumsum()

In [163]:
vol = 0.45

In [189]:
ps['cumsum2'] = ps['cumsum']-ps['size']

In [174]:
ps2 = ps[ps['cumsum2'] < vol]

In [178]:
ps2.shape

(3, 4)

In [180]:
ps2

,price,size,cumsum,cumsum2
0,110.0,0.1,0.1,0.0
1,111.0,0.2,0.3,0.1
2,112.0,0.4,0.7,0.3


In [182]:
ps2.at[ps2.shape[0]-1,'size'] = vol-ps2.at[ps2.shape[0]-1,'cumsum2']

In [183]:
ps2

,price,size,cumsum,cumsum2
0,110.0,0.10,0.1,0.0
1,111.0,0.20,0.3,0.1
2,112.0,0.15,0.7,0.3


In [79]:
type(ps.loc[0,'price'])

numpy.float64

In [85]:
import copy

In [106]:
import numpy as np

In [114]:
ps['cumsum'] = ps['size'].cumsum()
positions_for_income = ps[ps['cumsum'] <= 0.17].reset_index(drop=True)
price = positions_for_income['price'].values
size = positions_for_income['size'].values
ps = ps[ps['cumsum'] > 0.17].reset_index(drop=True)
price = np.append(price,ps['price'].values[0])
size = np.append(size,(0.17-positions_for_income['cumsum'].values[-1]))

In [115]:
price

array([110., 110.])

In [116]:
size

array([0.1 , 0.07])

In [102]:
price = ps['price'].values
size=  ps['size'].values
price

,price,size,cumsum
0,110.0,0.1,0.1
1,110.0,0.1,0.2
2,110.0,0.1,0.3


In [88]:
ps['cumsum'] = ps['size'].cumsum()
positions_for_income = ps[ps['cumsum'] <= 0.17].reset_index(drop=True)
ps = ps[ps['cumsum'] > 0.17].reset_index(drop=True)
tail = ps.head(1)
tail.loc[0,'size'] = 0.17 - (tail.loc[0,'cumsum']-tail.loc[0,'size'])
positions_for_income = positions_for_income.append(tail,ignore_index=True)

<ipython-input-88-c47ece35b68e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail.loc[0,'size'] = 0.17 - (copy.deepcopy(tail.loc[0,'cumsum'])-copy.deepcopy(tail.loc[0,'size']))


In [77]:
positions_for_income

,price,size,cumsum
0,110.0,0.10,0.1
1,110.0,0.07,0.2


In [60]:
ps.tail(1)

,price,size,cumsum
2,110.0,0.1,0.3


In [23]:
ps = ps[ps['cumsum'] > 0.17].reset_index(drop=True)

In [24]:
ps.loc[0,'size'] = ps.loc[0,'cumsum'] - 0.17

In [26]:
ps = ps[['price','size']]

In [33]:
ps['price2'] = ps['price']-100

In [34]:
ps

,price,size,price2
0,110.0,0.1,10.0
1,110.0,0.1,10.0
2,110.0,0.1,10.0


In [35]:
ps['reward'] = ps['size']*ps['price2']

In [37]:
ps['reward'].sum()

3.0

In [42]:
(1-ps['price']) * (1 if False  else -1)

0    109.0
1    109.0
2    109.0
Name: price, dtype: float64

In [320]:
pyb = pybitflyer.API(api_key=key,api_secret=secret)

In [326]:
r1 = pyb.getboardstate(product_code='FX_BTC_JPY')
r2 = pyb.getboardstate(product_code='FX_BTC_JPY')

In [327]:
r1

{'health': 'NORMAL', 'state': 'RUNNING'}

In [328]:
r2

{'health': 'NORMAL', 'state': 'RUNNING'}

In [329]:
result = pyb.sendchildorder(
    product_code="FX_BTC_JPY",
    child_order_type="MARKET",
    side="BUY",
    size=0.01,
    minute_to_expire=4,
    time_in_force="GTC"
)
result2 = pyb.sendchildorder(
    product_code="FX_BTC_JPY",
    child_order_type="MARKET",
    side="SELL",
    size=0.01,
    minute_to_expire=4,
    time_in_force="GTC"
)
print(result,result2)

{'child_order_acceptance_id': 'JRF20200920-144610-204804'} {'child_order_acceptance_id': 'JRF20200920-144610-003513'}


In [194]:
result

{'child_order_acceptance_id': 'JRF20200916-043754-126046'}

In [125]:
result.keys()

dict_keys(['child_order_acceptance_id'])

In [126]:
df = pd.DataFrame()

In [129]:
df['flag'] = [0,1,2,3]

In [130]:
df['size'] = [0,0,0,0]

In [149]:
df.loc[df['flag'] == 2]['size'] = 2

<ipython-input-149-60887771d3a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['flag'] == 2]['size'] = 2


In [134]:
df[df['flag'] == 2]

,flag,size
2,2,0


In [154]:
df.loc[df['flag'] == 2,'size'] = 2

In [155]:
df

,flag,size
0,0,0
1,1,0
2,2,2
3,3,0


In [292]:
import random
import math
round(random.uniform(0.01,0.015),3)

0.013

In [293]:
math.isclose(0.1,0.1)

True

In [294]:
math.isclose([1,2],[1,2])

TypeError: must be real number, not list

In [297]:
np.isclose([1,2],[1,2])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [299]:
np.logical_not(np.isclose([1,2],[1,2]))

array([False, False])

In [300]:
import xgboost

In [301]:
!ls

__pycache__                    reinforcement_q_learning.ipynb
backtest.py                    rl.ipynb
bcfx-Copy1.ipynb               scrap.ipynb
bcfx.ipynb                     skynet
bot.py                         ticker_history.ipynb
nn.ipynb                       xgboost.ipynb


In [302]:
import sys
sys.path.append("./skynet/bf/bf/scripts/")

In [304]:
import utils
import xgboost_ohlc

In [306]:
ohlc = utils.get_ohlc(periods=300,datasize=5000)
ohlc, data_columns, target_columns = utils.shaping_ohlc(ohlc,drop=False)
xgbc = xgboost_ohlc.c_fit(ohlc[data_columns],ohlc['Class'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  2.1min finished


In [307]:
ohlc

,timestamp,Open,High,Low,Close,O/O,H/O,L/O,C/O,O/C,...,Target_Low,Target_Close,TO/O,TH/O,TL/O,TC/O,TH/C,TL/C,TC/C,Class
0,1.598737e+09,1231710.0,1231888.0,1231173.0,1231377.0,1.0,1.000145,0.999564,0.999730,1.000270,...,1231000.0,1231815.0,0.999722,1.000257,0.999424,1.000085,1.000527,0.999694,1.000356,1
1,1.598738e+09,1231368.0,1232026.0,1231000.0,1231815.0,1.0,1.000534,0.999701,1.000363,0.999637,...,1231176.0,1231315.0,1.000369,1.000850,0.999844,0.999957,1.000487,0.999481,0.999594,0
2,1.598738e+09,1231822.0,1232415.0,1231176.0,1231315.0,1.0,1.000481,0.999476,0.999588,1.000412,...,1231083.0,1231185.0,0.999587,0.999739,0.999400,0.999483,1.000150,0.999812,0.999894,0
3,1.598738e+09,1231313.0,1231500.0,1231083.0,1231185.0,1.0,1.000152,0.999813,0.999896,1.000104,...,1230863.0,1231904.0,0.999903,1.000480,0.999635,1.000480,1.000584,0.999738,1.000584,1
4,1.598738e+09,1231194.0,1231904.0,1230863.0,1231904.0,1.0,1.000577,0.999731,1.000577,0.999424,...,1230306.0,1230400.0,1.000620,1.000682,0.999279,0.999355,1.000106,0.998703,0.998779,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4966,1.600236e+09,1144483.0,1144943.0,1143634.0,1144252.0,1.0,1.000402,0.999258,0.999798,1.000202,...,1144120.0,1144909.0,0.999773,1.000838,0.999683,1.000372,1.001040,0.999885,1.000574,1
4967,1.600236e+09,1144223.0,1145442.0,1144120.0,1144909.0,1.0,1.001065,0.999910,1.000600,0.999401,...,1143517.0,1143672.0,1.000663,1.001050,0.999383,0.999518,1.000451,0.998784,0.998920,0
4968,1.600236e+09,1144982.0,1145425.0,1143517.0,1143672.0,1.0,1.000387,0.998721,0.998856,1.001145,...,1143435.0,1143571.0,0.998881,0.999140,0.998649,0.998768,1.000284,0.999793,0.999912,0
4969,1.600237e+09,1143701.0,1143997.0,1143435.0,1143571.0,1.0,1.000259,0.999767,0.999886,1.000114,...,1142779.0,1143375.0,0.999888,1.000227,0.999194,0.999715,1.000341,0.999307,0.999829,0


In [308]:
xgbc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.05, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0500000007, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [316]:
class Fish(object):
    def __init__(self,name):
        self.name = name
        
class Kingyo(Fish):
    def call(self):
        print(self.name)
        print(self.name)

In [310]:
f = Fish('sakana')

In [312]:
f.name

'sakana'

In [317]:
ff = Kingyo('sakana')

In [318]:
ff.name

'sakana'

In [319]:
ff.call()

sakana
sakana


In [339]:
a = pd.DataFrame([])

In [340]:
a['price'] = [1,2,3,4,5]
a['size'] = [10,9,8,5,8]

In [345]:
a.at[a['size'].idxmax(),'price']

1

In [346]:
a

,price,size
0,1,10
1,2,9
2,3,8
3,4,5
4,5,8


In [347]:
pyb

In [348]:
last = 1104500
result = pyb.sendparentorder(
      order_method =  "IFDOCO",
      minute_to_expire = 100,
      time_in_force = "GTC",
      parameters = [{
        "product_code": "FX_BTC_JPY",
        "condition_type": "LIMIT",
        "side": "BUY",
        "price": last+200,
        "size": 0.01
      },
      {
        "product_code": "FX_BTC_JPY",
        "condition_type": "LIMIT",
        "side": "SELL",
        "price": int(last*1.0001),
        "size": 0.01
      },
      {
        "product_code": "FX_BTC_JPY",
        "condition_type": "STOP",
        "side": "SELL",
        "trigger_price": int(last*0.9999),
        "size": 0.01
      }]
    )

In [349]:
result

{'parent_order_acceptance_id': 'JRF20200922-103846-204880'}